# Trabalho Prático 2

## Introdução

A resolução deste trabalho prático tem como propósito introduzir o **SageMath**, corpos finitos primos, curvas elípticas sobre esses corpos e esquemas criptográficos baseados nos mesmos.
Os principais objetivos passam por:

- Implementar um esquema _RSA_ como uma classe **Python**.
- Implementar um esquema _ECDSA_ como uma classe **Python**.
- Implementar um esquema _ECDH_ como uma classe **Python**.

Além disso, devem ser aplicadas as seguintes particularidades aos esquemas desenvolvidos:

- O esquema _RSA_ deve fornecer métodos para cifrar, decifrar, assinar e verificar uma mensagem.
- O esquema _ECDSA_ deve utilizar uma das curvas primas definidas no **FIPS186-4**.
- O esquema _ECDH_ deve utilizar curvas elíticas binárias.

O relatório está dividido em três partes, uma por cada um dos objetivos a cumprir. Além disso, está estruturado de forma a que o texto entre os _snippets_ de código seja suficientemente explicativo, sobre a implementação e desenho da solução.

**PODERÁ SER PRECISO ALGO AQUI, COMO IMPORTS DO SAGEMATH OU ASSIM**

## Esquema RSA

O objetivo desta secção passa por definir a classe **Python** que implementa o algoritmo **RSA**. Esta permitirá inicializar uma intância, fornecendo-lhe o parâmetro de segurança (tamanho de chave). Após criada a instância, a mesma permitirá:

- Cifrar uma mensagem.
- Decifrar uma mensagem previamente cifrada.
- Assinar digitalmente uma mensagem.
- Verificar uma assinatura digital de uma mensagem, previamente produzida.

### Definição do esquema RSA

In [1]:
import hashlib
from sage.crypto.util import ascii_to_bin, bin_to_ascii

class RSA:
    
    def __init__(self,l):
        r = random_prime(2**((l/2) + 1)/2,True,(2**(l/2))/2)
        p = random_prime((2*r + 1)**2,True, 2*r + 1)
        self.q = p * r
        phi = (p - 1) * (r - 1)
        
        k = ZZ.random_element(phi)
        while gcd(k, phi) != 1:
             k = ZZ.random_element(phi)
        self.k = k
        
        self.s = inverse_mod(self.k,phi)
    
    def encrypt(self,plaintext):
        raw = ascii_to_bin(plaintext)
        plaintext = ZZ(int(str(raw),2))
        return power_mod(plaintext,self.k,self.q)       
    
    def decrypt(self,ciphertext):
        raw = power_mod(ciphertext,self.s,self.q)
        raw_b = raw.binary()
        
        #Acrescentar bits 0 menos significativos para que seja multiplo de 8
        bits_missing = 8 - Mod(len(raw_b),8)
        raw = ('0' * bits_missing) + raw_b

        return bin_to_ascii(raw)
    
    def hash(self, message):
        hash_context = hashlib.sha256(message).hexdigest()
        return hash_context
        
    
    def sign(self,message):
        msg_hash = self.hash(message)
        raw = ascii_to_bin(msg_hash)
        message = ZZ(int(str(raw),2))
        return power_mod(message,self.s,self.q)        
    
    def verify(self,message,signature):
        mk = power_mod(signature,self.k,self.q)
        mk_raw = mk.binary()
        
        #Acrescentar bits 0 menos significativos para que seja multiplo de 8
        bits_missing = 8 - Mod(len(mk_raw),8)
        mk_raw = ('0' * bits_missing) + mk_raw
        
        message = self.hash(message)
        mk_raw = bin_to_ascii(mk_raw)
        return mk_raw == message

### Teste ao Esquema RSA

In [2]:
r = RSA(2048)
msg = "Estruturas Criptográficas - Trabalho 2 - Iniciação SageMath - Esquema RSA"
print(msg)
ciphertext = r.encrypt(msg)

print(ciphertext)

plaintext = r.decrypt(ciphertext)
print(plaintext)

sig = r.sign(msg)
if(r.verify(msg,sig)):
    print('OK')
else: print('Not OK!')

Estruturas Criptográficas - Trabalho 2 - Iniciação SageMath - Esquema RSA
1205476194486712495392984932084025593918836594167186910792914193785929149741658224969215856620926130417391739300461886928650266346578218613215915900343651907029099608790373829678823285853530487629632927003969496276671806492804864739561760128077203563077884016283462206050756206461510251758254235204011510741781918339654311185921782177908918041049981186186597639380998545736843022886085234446719032836593745664247520614984835193083838623878373088228773505553651859813274284923931606223709430556124762488687633130297917108013410238395222518876614370238145705474498464861849168565569551449923459738989973475246405403135170203784736618179941332297017300872754880135640353745318216285661614309059023009201258177325360651519805286533669436674925753749951760397071243064118883371216385335290872494547838098589932028749739217773279546557101723053431581189268222398044133924402338432720488730897602907916654494334212159140491737337376


## Esquema ECDSA

### Definição do esquema ECDSA

In [75]:

class ECDSA:
    
    def __init(self,?):
    
    def sign(self,message):
    
    def verify(self,message,signature):

SyntaxError: invalid syntax (<ipython-input-75-3638a8099f06>, line 4)

### Teste ao esquema ECDSA

## Esquema ECDH

### Definição do esquema ECDH

### Teste ao esquema ECDH

## Conclusão

## Referências